In [22]:
import requests
import trafilatura
from bs4 import BeautifulSoup

def extraer_texto(url):
    try:
        res = requests.get(url, timeout=3)

        if res.status_code not in [200, 2001]:
            return None
        
        html = res.text
        resultado = trafilatura.extract(html,include_comments=False, include_tables=False)
        
        if not resultado:
            return None
        return resultado

    except:
        return None

def extraer_texto_mal(url):
    try:
        res = requests.get(url, timeout=3)
        if res.status_code != 200 or 'text/html' not in res.headers.get('Content-Type', ''):
            return None
        soup = BeautifulSoup(res.content, 'html.parser')
        parrafos = soup.find_all('p')
        texto = ' '.join([p.get_text(strip=True) for p in parrafos if p.get_text(strip=True)]) or None
        return texto
    except Exception:
        return None



In [23]:
import pandas as pd

df = pd.read_csv('./Data/allsides_links_def.csv')
df = df.drop_duplicates(subset='url')
df = df.dropna()
df = df[~df['bias'].isin(['allsides-new-12_0', 'not-yet-rated'])]
df = df.reset_index(drop=True)

df.head()


,url,topic,date,title,site,bias
0,https://www.npr.org/2021/01/01/952336030/what-...,general-news,2021-01-01,"What Got Us Through 2020? For Many, It Was Hob...",NPR (Online News),leaning-left
1,https://www.washingtonpost.com/opinions/2020/1...,general-news,2021-01-01,"Goodbye, 2020. You were awful, but some lights...",Guest Writer - Right,leaning-right
2,https://www.cbsnews.com/news/new-years-eve-gat...,general-news,2020-12-31,New Year's Eve gatherings could accelerate COV...,CBS News (Online),leaning-left
3,https://www.ibtimes.com/nye-traditions-you-can...,general-news,2020-12-31,NYE Traditions That You Can Still Participate ...,International Business Times,center
4,https://nypost.com/2020/12/31/heres-how-countr...,general-news,2020-12-31,Here’s how countries around the world are ring...,New York Post (News),leaning-right


In [38]:
import textwrap

url = df['url'][2]
print(url)

print()

texto = extraer_texto(url)
print(texto)

print('\n TEXT MAL FORMADO \n')

texto_mal = extraer_texto_mal(url)
print(textwrap.fill(texto_mal, width=140))

https://www.cbsnews.com/news/new-years-eve-gatherings-covid-19-spread/

New Year's Eve gatherings could accelerate COVID-19 spread: "We cannot let our guard down"
With New Year's Eve just one day away, health officials are sounding the alarm about large gatherings to ring in 2021. Their warnings come as the famous ball drop in New York's Times Square and many other big celebrations will be virtual this year.
But gatherings at home and other places could accelerate the spread of COVID-19.
"It's very important to avoid large gatherings and small gatherings, and as we approach New Year's Eve and New Year's Day, we cannot let our guard down," Pennsylvania Secretary of Health Dr. Rachel Levine warned.
According to AAA, as many as 84.5 million Americans were expected to travel from December 23 to January 3 to celebrate the holidays and the New Year.
"It would be magical thinking to think we're not going to see a bump after all this travel around the holiday season with Christmas and New Year

In [39]:
from tqdm import tqdm


df['page_text'] = None
    
for i in tqdm(range(len(df)), desc="Procesando URLs"):
    try:
        if pd.isna(df.loc[i, 'page_text']):
            url = df.loc[i, 'url']
            if isinstance(url, str) and url.startswith('http'):
                texto = extraer_texto(url)
                df.at[i, 'page_text'] = texto
            else:
                df.at[i, 'page_text'] = None

        if i > 0 and i % 1000 == 0:
            backup_path = f'/Users/pablogandia/Desktop/backups/allsides_backup{i}.csv'
            df.to_csv(backup_path, index=False)

    except Exception as e:
        print(f"[ERROR en índice {i}] → {e}")

final_path = '/Users/pablogandia/Desktop/allsides_links_trafilatura.csv'
df.to_csv(final_path, index=False)
print(f"\n[GUARDADO FINAL] → {final_path}")


Procesando URLs: 100%|██████████| 22222/22222 [4:32:29<00:00,  1.36it/s]  



[GUARDADO FINAL] → /Users/pablogandia/Desktop/allsides_links_trafilatura.csv


In [3]:
import requests
import undetected_chromedriver as uc
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import trafilatura
import time
import pandas as pd
from tqdm import tqdm

def get_driver():
    options = uc.ChromeOptions()
    options.add_argument("--headless")
    options.add_argument("--start-maximized")
    options.add_argument("--disable-popup-blocking")

    prefs = {
        "profile.default_content_setting_values.popups": 0,
        "profile.default_content_settings.popups": 0,
        "profile.managed_default_content_settings.popups": 0,
    }
    options.add_experimental_option("prefs", prefs)

    driver = uc.Chrome(options=options)
    driver.set_page_load_timeout(4)
    wait = WebDriverWait(driver, 2)
    return driver, wait

def simple_try(url):
    try:
        response = requests.get(url, timeout=0.5)
        if response.status_code == 200:
            return response.text
    except requests.RequestException as e:
        pass
    return None

def driver_try(url):
    driver, wait = get_driver()
    try:
        driver.get(url)
        wait.until(
            EC.any_of(
                EC.presence_of_element_located((By.TAG_NAME, "article")),
                EC.presence_of_element_located((By.TAG_NAME, "main")),
                EC.presence_of_element_located((By.CLASS_NAME, "content")),
                EC.presence_of_element_located((By.CLASS_NAME, "container")),
                EC.presence_of_element_located((By.TAG_NAME, "body"))
            )
        )
        time.sleep(0.1)
        page_source = driver.page_source
        driver.quit()
        return page_source
    except Exception as e:
        print(f"Driver failed: {e}")

    driver.quit()
    return None

def AutoScraper(url):
    html = simple_try(url)

    if not html:
        html = driver_try(url)
    
    if html:
        try:
            resultado = trafilatura.extract(html,include_comments=False, include_tables=False)
            return resultado
        except:
            return None

    print('ERROR OBTENIENENDO EL TEXTO')
    return None

In [5]:
df2 = pd.read_csv('/Users/pablogandia/Desktop/ASIGNATURAS/IA SAMSUNG/Proyecto/Bias/Data/allsides_links_trafilatura.csv')

filtro = df2['page_text'].isna() & (df2['bias'] == 'center')

df_filtrado = df2[filtro]

df_unico = df_filtrado.drop_duplicates(subset='site').reset_index(drop=True)

print(len(df_filtrado))
print(len(df_unico))


4121
92


In [ ]:
validos = {}
cont = 0


for urls,site in zip(df_unico['url'], df_unico['site']):
    cont += 1
    try:
        print(f"Procesando {site} → {cont}/92")
        a = time.time()
        texto = AutoScraper(urls)
        b = time.time()
        if texto and len(texto) > 300:
            validos[site] = texto
        else:
            print(f"FALLIDO: {site} → {urls}\n")
    except:
        pass

In [7]:
len(validos)

47

In [9]:
from IPython.display import clear_output

si = []

for i in validos:
    clear_output(wait=True)

    print(i)
    print()
    print(validos[i])

    a = input()
    if a == '':
        si.append(i)
    else:
        pass

Responsible Statecraft

It was widely expected, including by the victors of the 2024 presidential election, that the Biden government would ramp up its efforts through the end of the year to make it as difficult and costly as possible for a new administration to pursue a different course on Ukraine.
U.S. officials recently described this initiative as “Trump-proofing” their approach, and the White House decision to reportedly greenlight Ukrainian ATACMS strikes within Russian territory shows just how far the Biden administration is willing to go to handcuff President-elect Donald Trump to its Ukraine policy.
The announcement was tempered by some hedging, with officials telling the Washington Post that the initial missile strikes will “focus on and around” Russia’s southeastern border region of Kursk, though they “could expand” in the future.
The decision was preceded by weeks of public insistence by White House spokesman John Kirby and others that ATACMS strikes inside Russia offer lim

In [11]:
for i in si:
    print(i)

International Business Times
Military Times
The Fulcrum
Financial Times
Investopedia
Barron's
Fort Worth Star-Telegram
Hindustan Times
Mark Cannizzaro
CoinDesk
Crunchbase News
Forbes
Albuquerque Journal
CNET
Tulsa World
St. Louis Post-Dispatch
Pete Weichlein
Unusual Whales
BBC News
Straight Arrow News
The Kansas City Star
Responsible Statecraft


In [12]:
filtro = df2['page_text'].isna() & (df2['bias'] == 'center') & (df2['site'].isin(si))

len(df2[filtro])

87

In [ ]:
df2 = pd.read_csv('/Users/pablogandia/Desktop/ASIGNATURAS/IA SAMSUNG/Proyecto/Bias/Data/allsides_links_trafilatura.csv')
tqdm.pandas()

filtro = df2['page_text'].isna() & (df2['bias'] == 'center') & (df2['site'].isin(si))

subset_idx = df2[filtro].head(300).index

def recuperar_si_nan(texto, url):
    if texto and isinstance(texto, str):
        return texto
    try:
        texto_nuevo = AutoScraper(url)
        if texto_nuevo:
            return texto_nuevo
    except Exception as e:
        print(f'Error al recuperar texto para {url}: {e}')
    return None

df2.loc[subset_idx, 'page_text'] = df2.loc[subset_idx].progress_apply(
    lambda row: recuperar_si_nan(row['page_text'], row['url']), axis=1
)

df2.to_csv(
    '/Users/pablogandia/Desktop/ASIGNATURAS/IA SAMSUNG/Proyecto/Bias/Data/allsides_links_trafilatura_UPDATED.csv',
    index=False
)

In [14]:
df = pd.read_csv('/Users/pablogandia/Desktop/ASIGNATURAS/IA SAMSUNG/Proyecto/Bias/Data/allsides_links_trafilatura_UPDATED.csv')
len(df)

22222

In [15]:
df.head()

,url,topic,date,title,site,bias,page_text
0,https://www.npr.org/2021/01/01/952336030/what-...,general-news,2021-01-01,"What Got Us Through 2020? For Many, It Was Hob...",NPR (Online News),leaning-left,"What Got Us Through 2020? For Many, It Was Hob..."
1,https://www.washingtonpost.com/opinions/2020/1...,general-news,2021-01-01,"Goodbye, 2020. You were awful, but some lights...",Guest Writer - Right,leaning-right,NaN
2,https://www.cbsnews.com/news/new-years-eve-gat...,general-news,2020-12-31,New Year's Eve gatherings could accelerate COV...,CBS News (Online),leaning-left,New Year's Eve gatherings could accelerate COV...
3,https://www.ibtimes.com/nye-traditions-you-can...,general-news,2020-12-31,NYE Traditions That You Can Still Participate ...,International Business Times,center,NYE Traditions That You Can Still Participate ...
4,https://nypost.com/2020/12/31/heres-how-countr...,general-news,2020-12-31,Here’s how countries around the world are ring...,New York Post (News),leaning-right,Here’s how countries around the world are ring...
